In [1]:
"""
Program: hw3.ipynb
Author: David Gray
Description: Homework 3: A KNN algorithm that uses Euclidean distance and weighted voting via an inverse function of distance to classify objects.
Extra Credit: Functions available for finding the optimal k-value efficiently.
"""

import pandas as pd
import numpy as np
import math

In [2]:
def inverseFunction(x):
    # Returns the inverse of x
    return 1/x

In [3]:
def distance(p, q):
    # Returns the Euclidean distance between two points
    distance = 0
    for i in range(1,len(p)):
        distance += (p[i]-q[i])**2
    return math.sqrt(distance)

In [4]:
def getNeighbors(D, k, t):
    # Returns the k nearest neighbors
    neighbors = []
    total = len(D)
    for index in range(total):
        d = D.loc[index]
        dist = distance(t,d)
        if len(neighbors) < k:
            neighbors.append({'obj':d, 'distance':dist})
        elif [dist < neighbor['distance'] for neighbor in neighbors]:
            # Replace the neighbor that has the greatest distance from t with d
            distance_array = np.array([n['distance'] for n in neighbors])
            farthest = distance_array.max()
            for i in range(len(neighbors)):
                if distance_array[i] == farthest:
                    far_n = i
            neighbors[far_n] = {'obj':d, 'distance':dist}
    return neighbors

In [5]:
def weightedVote(neighbors, t):
    # Returns the predicted classification based on weighted votes of nearests neighbors
#     distances = np.array([neighbor['distance'] for neighbor in neighbors])
#     div = distances.max()
    for neighbor in neighbors:
        # Compare using percents
#         neighbor['weight'] = abs((div - neighbor['distance'])/div)+1
        # Compare distance with 1/x
        neighbor['weight'] = inverseFunction(neighbor['distance'])
    cls_and_distance = pd.DataFrame([{'class':neighbor['obj'].loc['label'],'weight':neighbor['weight']} for neighbor in neighbors])
    weight_sums = cls_and_distance['weight'].groupby(cls_and_distance['class']).sum()
    max_vote = weight_sums.max()
    class_vote = weight_sums[weight_sums == max_vote].keys()[0]
    return class_vote

In [6]:
def KNN(D, k, t):
    # KNN algorithm
    neighbors = getNeighbors(D, k, t)
    predicted_class = weightedVote(neighbors, t)
    return predicted_class

In [7]:
def basicClassification():
    # Classification using KNN
    train_dataset = pd.read_csv("MNIST_train.csv")
    test_dataset = pd.read_csv("MNIST_test.csv")
    k=5
    sample_count = 0
    correct_count = 0
    print("k =", k)
    for index in range(len(test_dataset)):
        obj = test_dataset.loc[index]
        actual_class = obj.loc['label']
        predicted_class = KNN(train_dataset, k, obj)
        print("Desired class:",actual_class, "-- Computed class: ", predicted_class)
        sample_count += 1
        if actual_class == predicted_class:
            correct_count += 1
    accuracy = (correct_count/sample_count)*100
    print("Accuracy rate: ", accuracy, "%")
    print("Number of missclassified test samples:", sample_count-correct_count)
    print("Total number of test samples:", sample_count)
    print("------------------------------------")

In [8]:
def predictClass(obj_model, k):
    # Returns the predicted classification based on weighted votes of nearests neighbors
    neighbors = obj_model[:k]
    for n in neighbors:
        n['weight'] = inverseFunction(n['distance'])
    cls_and_distance = pd.DataFrame([{'class':neighbor['obj'].loc['label'],'weight':neighbor['weight']} for neighbor in neighbors])
    weight_sums = cls_and_distance['weight'].groupby(cls_and_distance['class']).sum()
    max_vote = weight_sums.max()
    class_vote = weight_sums[weight_sums == max_vote].keys()[0]
    return class_vote

In [9]:
def modifiedKNN(model, k):
    # Classification using KNN where distances have already been calculated
    sample_count = 0
    correct_count = 0
    print("k =", k)
    for index in range(len(model)):
        obj_model = model[index]
        obj = obj_model[0]['obj']
        actual_class = obj['label']
        predicted_class = predictClass(obj_model[1:], k)
        sample_count += 1
        if actual_class == predicted_class:
            correct_count += 1
    accuracy = (correct_count/sample_count)*100
    print("Accuracy rate: ", accuracy, "%")
    print("Number of missclassified test samples:", sample_count-correct_count)
    print("Total number of test samples:", sample_count)
    print("------------------------------------")

In [10]:
def sortArray(array):
    # Returns a sorted array
    for i in range(2,len(array[1:])):
        item_to_insert = array[i]
        j = i - 1
        while j >= 1 and array[j]['distance'] > item_to_insert['distance']:
            array[j + 1] = array[j]
            j -= 1
        array[j + 1] = item_to_insert
    return array

In [11]:
def build_model(train_D, test_D):
    # Calculates and stores distances based on KNN
    complete_model = []
    total = len(train_D)
    for index_t in range(len(test_D)):
        t = test_D.loc[index_t]
        model_per_obj = []
        model_per_obj.append({'obj':t})
        for index_d in range(total):
            d = train_D.loc[index_d]
            dist = distance(t,d)
            model_per_obj.append({'obj':d, 'distance':dist})
        sort_model_per_obj = sortArray(model_per_obj)
        complete_model.append(sort_model_per_obj)
    return complete_model

In [12]:
def optimizedKClassification():
    # Build model
    train_dataset = pd.read_csv("MNIST_train.csv")
    test_dataset = pd.read_csv("MNIST_test.csv")
    model = build_model(train_dataset, test_dataset)
    # Find optimized k-value
    k_range = round(math.sqrt(len(train_dataset)))
    if k_range%2 == 0:
        k_range-=1
    for K in range(1,k_range):
        modifiedKNN(model, K)

In [13]:
print("################### basicClassification() ###################\n")
basicClassification()
print("\n################### optimizedKClassification() ###################\n")
optimizedKClassification()

k = 5
Desired class: 0 -- Computed class:  0
Desired class: 0 -- Computed class:  0
Desired class: 0 -- Computed class:  0
Desired class: 0 -- Computed class:  0
Desired class: 0 -- Computed class:  0
Desired class: 1 -- Computed class:  1
Desired class: 1 -- Computed class:  1
Desired class: 1 -- Computed class:  1
Desired class: 1 -- Computed class:  1
Desired class: 1 -- Computed class:  1
Desired class: 2 -- Computed class:  8
Desired class: 2 -- Computed class:  2
Desired class: 2 -- Computed class:  2
Desired class: 2 -- Computed class:  6
Desired class: 2 -- Computed class:  2
Desired class: 3 -- Computed class:  9
Desired class: 3 -- Computed class:  3
Desired class: 3 -- Computed class:  3
Desired class: 3 -- Computed class:  3
Desired class: 3 -- Computed class:  3
Desired class: 4 -- Computed class:  4
Desired class: 4 -- Computed class:  4
Desired class: 4 -- Computed class:  4
Desired class: 4 -- Computed class:  4
Desired class: 4 -- Computed class:  9
Desired class: 5 --